In [44]:
import pygrib
import pandas as pd
import os
import requests
from datetime import datetime

# pygrib reference https://jswhit.github.io/pygrib/api.html?highlight=open#pygrib.open
# RAQDPS reference https://eccc-msc.github.io/open-data/msc-data/nwp_raqdps/readme_raqdps-datamart_en/

In [45]:
# CONFIG OPTIONS
delete_file_after = True # decides whether or not to delete the downloaded file after
file_type = "T12Z_MSC_RAQDPS_PM2.5_Sfc_RLatLon0.09_PT000H" # type of file for desired data
start_hour = "12" # model run start, in UTC [00, 12]
forecast = "000" # forecast hour [000, 001, 002, ..., 072]


In [46]:
# Get the current date
current_date = datetime.now()
current_year = f"{current_date.year:04d}" # year as the string YYYY
current_month = f"{current_date.month:02d}" # month as the string MM
current_day = f"{current_date.day:02d}" # day as the string DD

base_url = "https://dd.weather.gc.ca/model_raqdps/10km/grib2"
file_name = current_year + current_month + current_day + file_type + ".grib2"

# URL of the GRIB2 file
grib_url = base_url + '/' + start_hour + '/' + forecast + '/' + file_name
print(f"grib_url = {grib_url}")

# Download the file
response = requests.get(grib_url)

# Check if response was succesful
if response.status_code == 200:
    # Success - Save the file locally
    grib_file = file_name # saves the downloaded file with the same name as the grib file
    with open(grib_file, 'wb') as f:
        f.write(response.content)
    print(f"GRIB2 file successfully downloaded as {grib_file}")
else:
    # Error occured
    print(f"Failed to download the file. Status code is {response.status_code}")
    exit()

grib_url = https://dd.weather.gc.ca/model_raqdps/10km/grib2/12/000/20241009T12Z_MSC_RAQDPS_PM2.5_Sfc_RLatLon0.09_PT000H.grib2
GRIB2 file successfully downloaded as 20241009T12Z_MSC_RAQDPS_PM2.5_Sfc_RLatLon0.09_PT000H.grib2


In [47]:
# Open the GRIB2 file
grbs = pygrib.open(grib_file)

# Iterate over each GRIB message and print parameter information
for grb in grbs:
    print(f"Message {grb.messagenumber}:")
    print(f"  Parameter: {grb.parameterName}")
    print(f"  Level: {grb.level}")
    print(f"  Data Type: {grb.typeOfLevel}")
    print(f"  Description: {grb.shortName}")
    print(f"  Valid Time: {grb.validDate}")
    print("-" * 40)
    

Message 1:
  Parameter: Mass density (concentration)
  Level: 2
  Data Type: heightAboveGround
  Description: mdens
  Valid Time: 2024-10-09 12:00:00
----------------------------------------


In [48]:
# Open GRIB2 file
grbs = pygrib.open(grib_file)

# Extract data from the GRIB2 file (you can iterate over messages for multiple parameters)
# In this case the only parameter is Mass density (concentration)
grb = grbs.message(1)
print(f"parameter details: {grb}")  # Print out parameter details

# Extract data
cons = grb.values  # Gets value for concentration parameter
lats, lons = grb.latlons()  # Get latitude and longitude coordinates

# Flatten the arrays to put into a DataFrame
flattened_data = {
    'latitude': lats.flatten(),
    'longitude': lons.flatten(),
    'concentration': cons.flatten()
}
df = pd.DataFrame(flattened_data)

# Save to CSV file
csv_file = 'output.csv'
df.to_csv(csv_file, index=False)

print(f"Data successfully saved to {csv_file}")

# Deletes the downloaded GRIB2 file if delete_file_after is True
if delete_file_after:
    os.remove(grib_file)
    print(f"GRIB2 file {grib_file} successfully deleted")

parameter details: 1:Mass density:kg m**-3 (instant):rotated_ll:heightAboveGround:level 2 m:fcst time 0 hrs:from 202410091200
Data successfully saved to output.csv
GRIB2 file 20241009T12Z_MSC_RAQDPS_PM2.5_Sfc_RLatLon0.09_PT000H.grib2 successfully deleted
